#### Install Necessary Packages

In [108]:
#!conda install -c conda-forge beautifulsoup4 --yes
#!conda install -c conda-forge lxml --yes
#!conda install -c conda-forge requests --yes

#### Import Necessary Libraries

In [109]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import requests

#### Use BeautifulSoup to scrap information in the table from the Wikipedia page, and set up the dataframe containing all the information in the table

In [110]:
wiki_html = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(wiki_html, 'lxml')
# print(soup.prettify())
table = soup.find('table', class_='wikitable sortable')
table_columns = []
for th_txt in table.tbody.findAll('th'):
    table_columns.append(th_txt.text.rstrip('\n'))

toronto_df = pd.DataFrame(columns=table_columns) 

for row in table.tbody.findAll('tr')[1:]:
    row_data = []
    for td_txt in row.findAll('td'):
        row_data.append(td_txt.text.rstrip('\n'))
    toronto_df = toronto_df.append({table_columns[0]: row_data[0],
                                    table_columns[1]: row_data[1],
                                    table_columns[2]: row_data[2]}, ignore_index=True)
toronto_df.head(10)

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
9,M8A,Not assigned,Not assigned


####  Remove cells with a borough that is Not assigned

In [111]:
toronto_df.replace('Not assigned',np.nan, inplace=True)
toronto_df = toronto_df[toronto_df['Borough'].notnull()]
toronto_df.reset_index(drop=True, inplace=True)
toronto_df.head(10)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,NaN
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


#### If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

In [112]:
toronto_df['Neighbourhood'] = np.where(toronto_df['Neighbourhood'].isnull(),toronto_df['Borough'],toronto_df['Neighbourhood'])
toronto_df.head(10)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Queen's Park
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


#### Combine neighbourhoods in the same postal code.

In [113]:
toronto_df = toronto_df.groupby(['Postcode','Borough'], as_index=False).agg(lambda x: ','.join(x))
toronto_df.reset_index(drop=True, inplace=True)
toronto_df.head(10)

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park"
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge"
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff,Cliffside West"


#### Print the number of rows of the dataframe

In [114]:
toronto_df.shape[0]

103